In [1]:
# Import needed modules
import pandas as pd

# Import the files
weather_df = pd.read_csv("Resources/Canton_Ohio_Weather.csv")
cfs_df = pd.read_excel("Resources/cfs_data_Canton.xlsx")

In [2]:
# Convert dt_iso into datetime format and convert to America/New_York timezone
weather_df['dt_iso'] = pd.to_datetime(weather_df['dt_iso'], format='%Y-%m-%d %H:%M:%S %z UTC')
weather_df['dt_iso'] = weather_df['dt_iso'].dt.tz_convert("America/New_York")
weather_df['dt_iso'].head(3)

0   2014-12-31 19:00:00-05:00
1   2014-12-31 20:00:00-05:00
2   2014-12-31 21:00:00-05:00
Name: dt_iso, dtype: datetime64[ns, America/New_York]

In [3]:
# Create column to merge weather and cfs data on.  Since weather data has
# by the hour with minutes and seconds 00:00, that will need to be the same
# for CFS data.
weather_df['relation'] = weather_df['dt_iso'].dt.strftime("%Y-%m-%d %H:%M:%S")
weather_df['relation'].head(3)

0    2014-12-31 19:00:00
1    2014-12-31 20:00:00
2    2014-12-31 21:00:00
Name: relation, dtype: object

In [4]:
# Create column within CFS to merge weather with.  This returns 00:00 for 
# minutes and seconds to match weather format.
cfs_df['relation'] = cfs_df['CreateDatetime'].dt.strftime("%Y-%m-%d %H:00:00")

In [5]:
# Merge the dataframes together on relation table with weather on the out
merged_df = pd.merge(cfs_df, weather_df, on="relation", how="outer")
merged_df.head(3)

,CallID,Department,CallType,CreateDatetime,GeoFlag,AgencyType,relation,dt,dt_iso,timezone,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,6140391.0,Canton Police Department,Disturbance,2023-12-22 21:09:34.750,False,1.0,2023-12-22 21:00:00,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6140386.0,Canton Police Department,911 Hangup,2023-12-22 21:04:17.120,False,1.0,2023-12-22 21:00:00,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6140368.0,Canton Police Department,911 Hangup,2023-12-22 20:44:22.477,False,1.0,2023-12-22 20:00:00,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Create crime_data_df to only use needed columns
crime_data_df = merged_df[['CallID','CallType','CreateDatetime','relation','temp','dew_point','feels_like','temp_min','temp_max','pressure','humidity','wind_speed','weather_main','weather_description']]
crime_data_df.columns

Index(['CallID', 'CallType', 'CreateDatetime', 'relation', 'temp', 'dew_point',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity',
       'wind_speed', 'weather_main', 'weather_description'],
      dtype='object')

In [7]:
# Dropna if CallID is NA.  We will need to fill or drop weather data due to
# the time of collection being before 2024
crime_data_df = crime_data_df.dropna(subset=['CallID'], axis='rows')

In [8]:
crime_data_df.head()

,CallID,CallType,CreateDatetime,relation,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,weather_main,weather_description
0,6140391.0,Disturbance,2023-12-22 21:09:34.750,2023-12-22 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6140386.0,911 Hangup,2023-12-22 21:04:17.120,2023-12-22 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6140368.0,911 Hangup,2023-12-22 20:44:22.477,2023-12-22 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6140364.0,911 Hangup,2023-12-22 20:41:46.953,2023-12-22 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6140355.0,Theft,2023-12-22 20:27:56.443,2023-12-22 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Create additional fields for year, month, day etc.
crime_data_df['year'] = crime_data_df['CreateDatetime'].dt.year
crime_data_df['month'] = crime_data_df['CreateDatetime'].dt.month
crime_data_df['weekday'] = crime_data_df['CreateDatetime'].dt.weekday
crime_data_df['month_name'] = crime_data_df['CreateDatetime'].dt.month_name(locale = 'English')
crime_data_df['hour'] = crime_data_df['CreateDatetime'].dt.hour

In [10]:
# Drop NA from weather temps until we get it filled.
crime_data_df = crime_data_df.dropna(subset=["temp"],axis="rows")
crime_data_df.head()

,CallID,CallType,CreateDatetime,relation,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,weather_main,weather_description,year,month,weekday,month_name,hour
464,6137619.0,Suspicious Person,2023-12-19 18:32:55.377,2023-12-19 18:00:00,27.01,20.08,21.63,23.94,29.5,1029.0,72.0,4.61,Clouds,scattered clouds,2023,12,1,December,18
465,6137614.0,Security Check,2023-12-19 18:27:08.113,2023-12-19 18:00:00,27.01,20.08,21.63,23.94,29.5,1029.0,72.0,4.61,Clouds,scattered clouds,2023,12,1,December,18
466,6137610.0,Parking Complaint,2023-12-19 18:24:17.883,2023-12-19 18:00:00,27.01,20.08,21.63,23.94,29.5,1029.0,72.0,4.61,Clouds,scattered clouds,2023,12,1,December,18
467,6137604.0,Suspicious Person,2023-12-19 18:20:50.703,2023-12-19 18:00:00,27.01,20.08,21.63,23.94,29.5,1029.0,72.0,4.61,Clouds,scattered clouds,2023,12,1,December,18
468,6137594.0,Suspicious Person,2023-12-19 18:06:37.350,2023-12-19 18:00:00,27.01,20.08,21.63,23.94,29.5,1029.0,72.0,4.61,Clouds,scattered clouds,2023,12,1,December,18


In [11]:
# Exported combined data for preservation
# crime_data_df.to_excel("combined_data.xlsx",index=False, header=True)

In [12]:
# accidents_df = crime_data_df.loc[crime_data_df['CallType'].str.contains("Accident")]
# accidents_df.head()

In [13]:
crime_data_df.to_excel("combined_data2.xlsx",index=False, header=True)